In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM Customers
    where Country in ('Germany', 'France' ) or city ='Madrid'
    order by 2,1
    ''',
    con,
)

,CustomerName,Country,Address
0,Blondel père et fils,France,"24, place Kléber"
1,Bon app',France,"12, rue des Bouchers"
2,Du monde entier,France,"67, rue des Cinquante Otages"
3,Folies gourmandes,France,"184, chaussée de Tournai"
4,France restauration,France,"54, rue Royale"
5,La corne d'abondance,France,"67, avenue de l'Europe"
6,La maison d'Asie,France,1 rue Alsace-Lorraine
7,Paris spécialités,France,"265, boulevard Charonne"
8,Spécialités du monde,France,"25, rue Lauriston"
9,Victuailles en stock,France,"2, rue du Commerce"


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [29]:
tasks+=1
pd.read_sql("""
    SELECT * from (
        SELECT Country, count(*) 
        FROM Customers 
        GROUP BY 1
        ORDER BY 2 DESC)
    LIMIT 3
    """,
    con,
)

,Country,count(*)
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [81]:
tasks+=1
pd.read_sql(
    ''' -- не вижу в таблицах даты отправления, пусть будет дата заказа
    select shippername, orderdate from (
    select *, row_number() over(order by date(OrderDate)) as ordernum from orders
    ) t 
    join shippers using(shipperid)
    where ordernum = 10
    ''',
    con,
).head(50)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [107]:
tasks+=1
pd.read_sql(
    '''
    with maxorder as (
    select distinct orderid, max(orderprice) from (
        SELECT *
            , sum(price) over(partition by orderid) as orderprice 
        FROM orders 
        join orderdetails using(orderid) 
        join products using(productid)
    ) 
    )
    
    select price, productname 
    from maxorder join 
    orderdetails using(orderid) 
    join products using(productid)
    ''',
    con,
)

,Price,ProductName
0,45.60,Rössle Sauerkraut
1,123.79,Thüringer Rostbratwurst
2,263.50,Côte de Blaye
3,20.00,Maxilaku
4,7.45,Tourtière


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [ ]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [131]:
tasks+=1
pd.read_sql(
    '''
    with productids as ( 
    SELECT productid, count(*) FROM 
    orders 
    join orderdetails using(orderid)
    join products using (productid)
    group by productid
    order by 2 desc
    limit 1) --select * from productids
    
    select  productname, orderid, count(*) as cnt from 
    products
    join orderdetails using(productid) 
    join orders using(orderid)
    where productid in (select productid from productids)
    group by 1,2
    ''',
    con,
)

,ProductName,OrderID,cnt
0,Mozzarella di Giovanni,10248,1
1,Mozzarella di Giovanni,10268,1
2,Mozzarella di Giovanni,10269,1
3,Mozzarella di Giovanni,10272,1
4,Mozzarella di Giovanni,10274,1
5,Mozzarella di Giovanni,10283,1
6,Mozzarella di Giovanni,10297,1
7,Mozzarella di Giovanni,10325,1
8,Mozzarella di Giovanni,10330,1
9,Mozzarella di Giovanni,10337,1


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [ ]:
страну, контактное лицо и телефон.

In [181]:
pd.read_sql("""
    select Suppliername, ContactName, Country, Phone from (
    select Suppliername, ContactName, Country, Phone,count(*)  from (
    select distinct suppliername, ContactName, Country, Phone, orderid
    from Suppliers 
    join products using(supplierid)
    join orderdetails using(productid)  
    ) group by 1,2,3,4
    order by 5 desc
    limit 5
    )
    """, con)

,Suppliername,ContactName,Country,Phone
0,"Pavlova, Ltd.",Ian Devling,Australia,(03) 444-2343
1,Plutzer Lebensmittelgroßmärkte AG,Martin Bein,Germany,(069) 992755
2,"Specialty Biscuits, Ltd.",Peter Wilson,UK,(161) 555-4448
3,Norske Meierier,Beate Vileid,Norway,(0)2-953010
4,Formaggi Fortini s.r.l.,Elio Rossi,Italy,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [204]:
tasks+=1
pd.read_sql(
    '''
    SELECT country, categoryname, sum(price) 
    FROM Categories 
    join Products using(categoryid)
    join Orderdetails using(productid)
    join orders using(orderid)
    join Customers using(customerid)
    where country = 'Brazil'
    group by categoryname
    order by 3 desc
    limit 1
    ''',
    con,
)#['count(*)'].sum()

,Country,CategoryName,sum(price)
0,Brazil,Beverages,405.75


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [217]:
tasks+=1
pd.read_sql(
    '''
    select max(s) - min(s) as "разница в стоимости" from (
    SELECT *, sum(price) as s
    FROM Categories 
    join Products using(categoryid)
    join Orderdetails using(productid)
    join orders using(orderid)
    join Customers using(customerid)
    where country = 'USA'
    group by orderid)
    ''',
    con,
)#['count(*)'].sum()

,разница в стоимости
0,334.09


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [264]:
pd.read_sql(
    """
    select * from orders
    """,
    con,
)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,04.07.1996,3
1,10249,81,6,05.07.1996,1
2,10250,34,4,08.07.1996,2
3,10251,84,3,08.07.1996,1
4,10252,76,4,09.07.1996,2
...,...,...,...,...,...
191,10439,51,6,07.02.1997,3
192,10440,71,4,10.02.1997,2
193,10441,55,3,10.02.1997,2
194,10442,20,3,11.02.1997,2


In [272]:
tasks+=1
pd.read_sql(
    """
    with e as (select * from (
    SELECT *, 
        --strftime('%d.%m.%Y',birthdate) as birthdate 
        -- suka ne rabotaet
        substring(birthdate, 7, 5) || '-' || substring(birthdate, 4, 2) || '-'  || substring(birthdate, 1, 2) as dt 
    FROM employees ) order by dt desc limit 3)
    
    select LastName || ' ' || FirstName as name, count(*) as "количество заказов" 
    from e join orders using(EmployeeID) group by 1
    """,
    con,
)

,name,количество заказов
0,Davolio Nancy,29
1,Dodsworth Anne,6
2,Leverling Janet,31


10. Сколько банок крабового мяса всего было заказано.

In [274]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM Products where productname like '%crab%'
    ''',
    con,
)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,40,Boston Crab Meat,19,8,24 - 4 oz tins,18.4


предположим это то что мы ищем

In [279]:
tasks+=1
pd.read_sql(
    '''
    SELECT count(*) as 'количество банок крабового мяса' FROM  orders join orderdetails using(orderid) where productid = 40
    ''',
    con,
)

,количество банок крабового мяса
0,8


In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [281]:
tasks = 10

In [282]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
